# Perdido en el medio: El problema con los contextos largos

"Independientemente de la arquitectura de tu modelo, existe una degradación sustancial del rendimiento cuando incluyes más de 10 documentos recuperados. En resumen: Cuando los modelos deben acceder a información relevante en medio de contextos largos, tienden a ignorar los documentos proporcionados. Ver: https://arxiv.org/abs/2307.03172

Para evitar este problema, puedes reordenar los documentos después de recuperarlos para evitar la degradación del rendimiento."

---
[Langchain](https://python.langchain.com/docs/modules/data_connection/document_transformers/post_retrieval/long_context_reorder)

## Librerías

In [ ]:
from operator import itemgetter

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_transformers import LongContextReorder
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import LangchainDocsLoader, num_tokens_from_string

load_dotenv()

## Carga de datos

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=10,
    length_function=num_tokens_from_string,
)

documents = LangchainDocsLoader().load()
documents = text_splitter.split_documents(documents)

## Creación de retriever

In [ ]:
retriever = Chroma.from_documents(documents, embedding=OpenAIEmbeddings()).as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "fetch_k": 50,
    },
)

## Consulta con el retriever

In [ ]:
relevant_docs = retriever.get_relevant_documents(
    "How to use LCEL ainvoke with a retriever?"
)
relevant_docs

## Reordenado de documentos

## Uso del reordenador en nuestro pipeline de `Retrival Augmented Generation`

In [ ]:
def combine_documents(documents: list[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in documents])


prompt = PromptTemplate.from_template(
    """Given the following text extracts:
-----
{context}
-----
                                      
Answer the following question, if you don't know the answer, just write "I don't know.

Question: {question}"""
)

llm = ChatOpenAI(temperature=0)

stuff_chain = (
    {
        "context": itemgetter("question")
        | retriever
        | combine_documents,
        "question": itemgetter("question"),
    }
    | prompt
    | llm
)

In [ ]:
response = stuff_chain.invoke(input={"question": "How to create a chain using LCEL?"}).content
print(response)